In [1]:
import MDAnalysis as mda
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from os.path import join
from glob import glob
from matplotlib import pyplot as plt
from MDAnalysis import transformations
from decimal import Decimal

from matplotlib import cm 
from matplotlib.colors import ListedColormap,LinearSegmentedColormap
import matplotlib.colors
import seaborn as sns

from itertools import islice
import glob
import shutil

/u/adlouet/conda-envs/tunnel/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','')

In [3]:
# The following path shouldn't be touched
prepared_ff=('//ptmp/adlouet/alchem/lambda/input_files/prepared_files')

##### This path needs to be altered and needs the following files in directory : tpr, (1 full length) xtc, sol.txt, residuetypes.dat


In [4]:
path=("//ptmp/adlouet/alchem/testing_method/ASASA/A_SP1_A_SP2_A/rep_0")

## Pre-Processing on original Conformation

This bash script prints out a new gro file every 10 frames (gmx trjconv -sep command), and creates a new file for each new gro. It then adds the preperared_files ff in each pair file, and changes the solvent and mol size so that it correpsonds with what is currently in the gro file.  

In [ ]:
%%bash -s "$prepared_ff" "$path"

module load gromacs/2021.5

cd $2

echo 0 |gmx trjconv -f *.xtc -s *.tpr -skip 10 -sep -o pair.gro

for x in ./pair*; do
mkdir "${x%.*}" && mv "$x" "${x%.*}"
done

#cp -r //ptmp/adlouet/alchem/lambda/input_files/prepared_files .

cp -r $1 .

for x in ./pair*; do
(cd ${x} && ln -s ../prepared_files .)
done

cd prepared_files
for x in lambda_{0..10}; do (cd ${x} && cat ../../sol.txt >> *.top); done
cd ../

This calls another python script to go into gro file and change atom # on line two and adds the new ZH (dummy) atom in SP2

In [ ]:
for filename in glob.iglob(f'{path}/pair*'):
    #print (filename)
    %run adding_H-Copy1.ipynb

    change_atom_number(filename)
    #x=filename
    

The following script produces 10 new 2 step trajectories for lambda 0 --> lambda 1.0. Lambda 0 is SP1-SP2 (original form) to lambda SP1-SP2 (with SP1 H 0% electrostic interactions? and SP2 100% electrostatic interacitions?). This allows the addition of new H atom on SP2 and disappearence of H on SP1 without causing any steric clashing bc the addition of atoms is very gradual (10% at a time).

##### This process produces a lot of files and takes a long time. Best executed with command jupyter nbconvert --to notebook --execute pre_processing1_0.ipynb --ExecutePreprocessor.timeout=-1 on cluster


In [ ]:
%%bash -s "$path"

module load gromacs/2021.5
cd $1   
array=(   {0..10} )

array2=(   {z,0,1,2,3,4,5,6,7,8,9,10} )

for x in pair*; do (cd ${x} &&

    for index in ${!array[*]}; do 
      gmx grompp -f prepared_files0_1/lambda_"${array[$index]}"/*.mdp -c out_lam_"${array2[$index]}".gro -p prepared_files0_1/lambda_"${array[$index]}"/*.top -o topol_"${array[$index]}".tpr -maxwarn 20
      gmx mdrun -s topol_"${array[$index]}".tpr
      mv confout.gro out_lam_"${array[$index]}".gro
      mv md.log md_"${array[$index]}".log;
      rm \#*
        #echo "${array[$index]} is in ${array2[$index]}"
    done);
    
done
    
    
    

Can now run the post-processing script to extract PE information from each md.log file